In [10]:
import concurrent.futures
import pathlib
import os

import polars as pl
import plotnine as p9

from tools.sequence.sequence import Sequence

DATA_DIR = pathlib.Path(os.getcwd()) / ".." / "_analysis"
OUTPUT_DIR = DATA_DIR / "output"

In [11]:
df = pl.read_csv(OUTPUT_DIR / "1-optimise.csv")
df

source,name,input_sequence,output_sequence,failed_reason
str,str,str,str,str
"""hrt_hk_nt_seq_pulled_from_ense…","""ENSG00000100387|ENSG0000010038…","""ATGGCGGCAGCGATGGATGTGGATACCCCG…","""ATGGCCGCCGCCATGGACGTGGACACCCCC…",""""""
"""hrt_hk_nt_seq_pulled_from_ense…","""ENSG00000111639|ENSG0000011163…","""ATGGCAGGGAACCTCTTATCCGGGGCAGGT…","""ATGGCCGGCAACCTGCTGAGCGGCGCCGGC…",""""""
"""hrt_hk_nt_seq_pulled_from_ense…","""ENSG00000239789|ENSG0000023978…","""ATGTCCGTAGTTCGCTCATCCGTCCATGCC…","""ATGAGCGTGGTGAGAAGCAGCGTGCACGCC…",""""""
"""hrt_hk_nt_seq_pulled_from_ense…","""ENSG00000171148|ENSG0000017114…","""ATGAGTGAGTTGAAAGACTGCCCCTTGCAG…","""ATGAGCGAGCTGAAGGACTGCCCCCTGCAG…",""""""
"""hrt_hk_nt_seq_pulled_from_ense…","""ENSG00000169020|ENSG0000016902…","""ATGGTGCCACCGGTGCAGGTCTCTCCGCTC…","""ATGGTGCCCCCCGTGCAGGTGAGCCCCCTG…",""""""
…,…,…,…,…
"""ensembl_ccds_tx_nt_seq.fa""","""ENSG00000198300|ENSG0000019830…","""ATGTACCAACCAGAAGACGACAACAACAGT…","""ATGTACCAGCCCGAGGACGACAACAACAGC…",""""""
"""ensembl_ccds_tx_nt_seq.fa""","""ENSG00000162727|ENSG0000016272…","""ATGGCATGGGAGAATCAGACCTTCAACTCT…","""ATGGCCTGGGAGAACCAGACCTTCAACAGC…",""""""
"""ensembl_ccds_tx_nt_seq.fa""","""ENSG00000187514|ENSG0000018751…","""ATGTCAGACGCAGCCGTAGACACCAGCTCC…","""ATGAGCGACGCCGCCGTGGACACCAGCAGC…",""""""


In [12]:
def _mfe(sequence: str) -> float | None:
    if not sequence:
        return None

    return Sequence.from_string(sequence).minimum_free_energy.energy


with concurrent.futures.ProcessPoolExecutor() as executor:
    df = df.with_columns(
        input_mfe=pl.Series(list(executor.map(_mfe, df["input_sequence"].to_list()))),
        output_mfe=pl.Series(list(executor.map(_mfe, df["output_sequence"].to_list()))),
    )

df.write_csv(output_dir / "2.1-hydrate-mfe.csv")

NameError: name 'output_dir' is not defined

In [13]:
df.write_csv(OUTPUT_DIR / "2.1-hydrate-mfe.csv")